<a href="https://colab.research.google.com/github/yunyonggeun/MLforBioPh/blob/master/Colab_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 코랩에 대해 배우기

> 2020.1.22. WendesDay

> 학습 사이트 : 민석님의 페이지

In [2]:
import tensorflow as tf

In [3]:
import keras

Using TensorFlow backend.


In [0]:
import boto3

## Upload and Download

In [0]:
import pandas as pd

df = pd.read_csv("sample_data/california_housing_test.csv")

In [6]:
df.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


## GPU 사용하기



In [5]:
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [8]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.0320500220000213
GPU (s):
0.042170819999910236
GPU speedup over CPU: 71x


https://colab.research.google.com/github/yunyonggeun/MLforBioPh/blob/master/01_Study_Numpy.ipynb


> Github의 파일을 직접 내 colab에서 열기를 위해서는 아래와 같이 링크를 걸어 줌

[![Open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yunyonggeun/MLforBioPh/blob/master/01_Study_Numpy.ipynb)

# 구글 드라이브와 연동하기

> 구글 드라이브 해당 폴더의 아이디를 확인해 둠

> 예) 1fz0FWiO05diIX636uDwxbi9yHbFwOoQ7

+ 구글 드라이브를 이용하기 위한 라이브러리 불러오기

In [0]:
!pip install -U -q PyDrive

+ 접근 권한을 받기

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
   
# PyDrive Authentication
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

+ 폴더 아이디로 File List 받아오기

In [0]:
folder_id = '1fz0FWiO05diIX636uDwxbi9yHbFwOoQ7'
   
def ListFolder(parent):
    filelist=[]
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
    for f in file_list:
        if f['mimeType']=='application/vnd.google-apps.folder': # if folder
            filelist.append({"id":f['id'],"title":f['title'],"list":ListFolder(f['id'])})
        else:
            filelist.append({"title":f['title'],"id":f['id']})
    return filelist
   
   
file_lists = ListFolder(folder_id)

+ 로컬 디렉토리에 저장

In [0]:
# choose a local (colab) directory to store the data.
import os
local_download_path = os.path.expanduser('~/my_sample_data')
try:
	os.makedirs(local_download_path)
except: pass

In [17]:
ret = []
for file in file_lists:
    print('title: %s, id: %s' % (file['title'], file['id']))
    fname = os.path.join(local_download_path, file['title'])
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': file['id']})
    f_.GetContentFile(fname)
    print(fname)
    ret.append(fname)

title: california_housing_test.csv, id: 1S43vjNOBxVIARmUbP4IgKiVbjNzZMI7d
downloading to /root/my_sample_data/california_housing_test.csv
/root/my_sample_data/california_housing_test.csv
title: colab_test.ipynb, id: 1jedA4fNT2lRAtPkoHT7099J4Vg0qa5j_
downloading to /root/my_sample_data/colab_test.ipynb
/root/my_sample_data/colab_test.ipynb


In [18]:
ret

['/root/my_sample_data/california_housing_test.csv',
 '/root/my_sample_data/colab_test.ipynb']

In [16]:
import pandas as pd
pd.read_csv(ret[0])

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23.0,1450.0,642.0,1258.0,607.0,1.1790,225000.0
2996,-118.14,34.06,27.0,5257.0,1082.0,3496.0,1036.0,3.3906,237200.0
2997,-119.70,36.30,10.0,956.0,201.0,693.0,220.0,2.2895,62000.0
2998,-117.12,34.10,40.0,96.0,14.0,46.0,14.0,3.2708,162500.0
